In [1]:
import pandas as pd

In [2]:
csv = pd.read_csv("../complete_income_statements.csv",keep_default_na=True,na_values=["None", "none", "NaN", "-"],low_memory=False,dtype={"company_id": str})
csv["company_id"] = csv["company_id"].astype(str)

In [3]:
df = csv
bad_ids = df[~df["company_id"].str.fullmatch(r"\d{4}")]

In [4]:
# Ensure company_id is string
csv["company_id"] = csv["company_id"].astype(str)

# Detect bad IDs (not exactly 4 digits)
bad_mask = ~csv["company_id"].str.fullmatch(r"\d{4}")

# Fix only the bad ones
csv.loc[bad_mask, "company_id"] = csv.loc[bad_mask, "company_id"].str.zfill(4)

In [5]:
df = csv

In [6]:
# Step 1: Separate Value and YoY rows
df_value = df[df["Year/Type"].str.contains("Value", na=False)].copy()
df_yoy = df[df["Year/Type"].str.contains("YoY %", na=False)].copy()

# Step 2: Extract full date from "Year/Type" (e.g., '31 Mar 2024')
df_value["Fiscal Date"] = df_value["Year/Type"].str.extract(r"(\d{1,2} \w{3} \d{4})")
df_yoy["Fiscal Date"] = df_yoy["Year/Type"].str.extract(r"(\d{1,2} \w{3} \d{4})")

# Step 3: Merge on 'company_id' and 'Fiscal Date'
merged = pd.merge(
    df_value,
    df_yoy,
    on=["company_id", "Fiscal Date"],
    suffixes=("", "_yoy"),
    how="left"
)

# Drop helper columns
merged = merged.drop(columns=["Year/Type", "Year/Type_yoy"])
merged = merged.rename(columns={"Fiscal Date": "Fiscal Date"})

In [7]:
# Find all _yoy columns
yoy_cols = [col for col in merged.columns if col.endswith('_yoy')]

# Identify _yoy columns that do NOT contain any '%' values
cols_to_drop = [
    col for col in yoy_cols
    if not merged[col].astype(str).str.contains('%', na=False).any()
]

# Drop them
merged = merged.drop(columns=cols_to_drop)


In [8]:
bursa_registration = pd.read_csv("../matched_companies_from_ssm.csv",dtype={"company_id": str,"companyNo": str})
bursa_registration["companyNo"] = (
    bursa_registration["companyNo"]
    .astype(str)
    .str.replace(r"\.0$", "", regex=True)
    .str.strip()
)




In [9]:
company_id = pd.read_csv("../../../list_bursa_ids/bursa_company_list.csv", dtype={"company_id": str})
company_id["company_id"] = company_id["company_id"].str.strip().str.zfill(4)


In [10]:
merge = pd.merge(bursa_registration,company_id,on="company_name",how="inner")

In [11]:
merge = merge.drop(columns="company_type")
merge = merge.rename(columns={
    "company_name": "company_name_bursa",
    "name_db": "company_name_api",
    "companyNo": "registration_number",
    "oldCompanyNo": "old_registration_number"
})

In [12]:
merge["company_id"] = merge["company_id"].str.strip()
merged["company_id"] = merged["company_id"].str.strip()

In [13]:
merge["company_id"] = merge["company_id"].astype(str)
merged["company_id"] = merged["company_id"].astype(str)
final_merge = merge.merge(merged, on="company_id", how="inner")

In [14]:
df=final_merge

In [15]:
# ── Your DataFrame ── ## Column Name cleanup

df.columns = df.columns.str.replace("-", " ").str.strip().str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")

df.columns = (
    df.columns
    .str.replace("-", " ", regex=False)        # Replace hyphens with spaces
    .str.replace("/", " ", regex=False) 
    .str.strip()
    .str.lower()
    .str.replace(r"[^\w\s]", "", regex=True)   # Remove non-word characters like . , &
    .str.replace(r"\s+", "_", regex=True)      # Convert spaces to single underscore
    .str.replace(r"_+", "_", regex=True)       # Collapse multiple underscores to one
)
# Check for duplicate column names
duplicates = df.columns[df.columns.duplicated()]

df = df.loc[:, ~df.columns.duplicated()]
# Drop all *_yoy columns before calculating null ratios
yoy_columns = [col for col in df.columns if col.endswith('_yoy')]
df = df.drop(columns=yoy_columns)
# Get null ratios sorted
null_ratios = df.isnull().mean().sort_values(ascending=False)

# Convert to DataFrame with formatted percentage
summary_df = pd.DataFrame({
    "column": null_ratios.index,
    "null_ratio": (null_ratios * 100).map("{:.2f}%".format)
})

# # Save to CSV
summary_df.to_csv("../../column_percentages/portal_inc_null_ratio_summary.csv", index=False)
desired_columns = [
    "company_id",  # ✅ Add this
    "registration_number",
    "old_registration_number",
    "revenue",
    "net_income_before_taxes",
    "net_income_after_taxes",
    "fiscal_date",
    "bank_total_revenue"
]

# Clean bank_total_revenue (remove commas, convert to float)
df["bank_total_revenue"] = (
    df["bank_total_revenue"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.strip()
    .pipe(pd.to_numeric, errors="coerce")
    .fillna(0)
)

# Replace revenue only if it's NaN or 0
df["revenue"] = df["revenue"].fillna(0)
df["revenue"] = df.apply(
    lambda row: row["bank_total_revenue"] if row["revenue"] == 0 and row["bank_total_revenue"] > 0 else row["revenue"],
    axis=1
)

# Drop the helper column
df.drop(columns=["bank_total_revenue"], inplace=True)


# Convert fiscal_date to datetime for sorting
df["fiscal_date"] = pd.to_datetime(df["fiscal_date"], format="%d %b %Y", errors="coerce")

# Sort descending by fiscal_date within each company_id
df = df.sort_values(by=["company_id", "fiscal_date"], ascending=[True, False])

# Convert back to string format expected by DB
df["fiscal_date"] = df["fiscal_date"].dt.strftime("%d %b %Y")


df = df[[col for col in desired_columns if col in df.columns]]
df = df.dropna(subset=["registration_number"])

## ALL THE BANKS don't use revenue in their financials statements
company_1155_manual_revenue = {
    "31 Dec 2024": 27907,
    "31 Dec 2023": 25650,
    "31 Dec 2022": 23702,
    "31 Dec 2021": 22249,
    "31 Dec 2020": 19670
}

for date, value in company_1155_manual_revenue.items():
    mask = (df["company_id"] == "1155") & (df["fiscal_date"] == date)
    df.loc[mask, "revenue"] = float(value)
#---- manual additions
cimb_manual_revenue = {
    "31 Dec 2024": 22301.154,
    "31 Dec 2023": 21014.482,
    "31 Dec 2022": 19837.516,
    "31 Dec 2021": 19512.940,
    "31 Dec 2020": 17189.003
}

for date, value in cimb_manual_revenue.items():
    mask = (df["company_id"] == "1023") & (df["fiscal_date"] == date)
    df.loc[mask, "revenue"] = float(value)
    
hlb_manual_revenue = {
    "30 Jun 2024": 5884,
    "30 Jun 2023": 5570,
    "30 Jun 2022": 5417,
    "30 Jun 2021": 4803,
    "30 Jun 2020": 4399
}

for date, value in hlb_manual_revenue.items():
    mask = (df["company_id"] == "5819") & (df["fiscal_date"] == date)
    df.loc[mask, "revenue"] = float(value)
    

public_bank_manual_revenue = {
    "31 Dec 2024": 14040,
    "31 Dec 2023": 12949,
    "31 Dec 2022": 13065,
    "31 Dec 2021": 11305,
    "31 Dec 2020": 10045
}
for date, value in public_bank_manual_revenue.items():
    mask = (df["company_id"] == "1295") & (df["fiscal_date"] == date)
    df.loc[mask, "revenue"] = float(value)





In [16]:
# Get null ratios sorted
null_ratios = df.isnull().mean().sort_values(ascending=False)

# Convert to DataFrame with formatted percentage
summary_df = pd.DataFrame({
    "column": null_ratios.index,
    "null_ratio": (null_ratios * 100).map("{:.2f}%".format)
})

# # Save to CSV
summary_df.to_csv("../../column_percentages/inc_portal_null_ratio_summary.csv", index=False)
summary_df.head()

,column,null_ratio
old_registration_number,old_registration_number,2.54%
net_income_after_taxes,net_income_after_taxes,2.49%
net_income_before_taxes,net_income_before_taxes,0.78%
company_id,company_id,0.00%
registration_number,registration_number,0.00%
